# Example 1: Get protein information, run BLAST, and dump the results into a database


In [7]:
%reload_ext autoreload
%autoreload 2

from pyEED.core.proteinsequence import ProteinSequence
from pyEED.ncbi.utils import get_nucleotide_sequences

## Query NCBI

The pyEED library is centered around the `ProteinSequence` object, which integrates available information on protein sequence, corresponding nucleotide sequence, as well as regions and sites within the sequences. The `ProteinSequence` can be initialized directly with a protein sequence accession number.

In [42]:
aldolase = ProteinSequence.from_ncbi("NP_001287541.1")
print(aldolase)

ProteinSequence
├── id = NP_001287541.1
├── name = aldolase 1, isoform M
├── sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = taxon:7227
│       ├── name = Drosophila melanogaster
│       └── taxonomy_id = 7227
├── regions
│   └── 0
│       └── Region
│           ├── id = region32
│           ├── start = 14
│           ├── end = 361
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │       ├── id = site6
│   │       ├── name = active
│   │       ├── type = active
│   │       ├── po

In [43]:
# Get the corresponding nucleotide sequence
aldolase.get_nucleotide_seq()
print(aldolase.coding_sequence)

NucleotideSequence
├── id = NM_001300612.1
├── regions
│   └── 0
│       └── Region
│           ├── id = NM_001300612.1
│           ├── start = 278
│           └── end = 1363
├── molecule_type = mRNA
├── gene_id = Ald1
└── sequence = ATGACGACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAAATCGCCCAGAAAATCGTTGCCCCCGGCAAGGGAATCCTCGCCGCCGATGAGTCCGGCCCAACCATGGGCAAGCGTCTGCAGGACATCGGCGTGGAGAACACCGAGGACAACCGCCGTGCCTACCGTCAGCTGTTGTTCAGCACTGACCCCAAGCTGGCCGAGAACATCTCTGGAGTGATCCTGTTCCACGAGACCCTCTACCAGAAGGCCGATGATGGCACCCCCTTCGCCGAGATCCTGAAGAAGAAGGGAATCATTCTGGGCATCAAGGTCGACAAGGGTGTTGTCCCACTGTTCGGCTCTGAGGATGAGGTCACCACCCAGGGTCTGGATGACCTGGCCGCCCGTTGCGCCCAGTACAAGAAGGACGGTTGCGACTTCGCCAAGTGGCGTTGCGTCCTGAAGATCGGCAAGAACACCCCATCCTACCAGTCGATCCTGGAGAACGCCAATGTCCTGGCCCGCTACGCCTCCATCTGCCAGTCGCAGCGCATCGTCCCAATTGTGGAGCCCGAGGTTCTGCCCGATGGCGATCACGATCTGGACCGCGCCCAGAAGGTCACCGAGACCGTCCTGGCCGCCGTCTACAAGGCCCTGAGCGACCACCACGTCTACCTGGAGGGTACTCTGCTGAAGCCCAACATGGTCACCGCCGGTCAGTCGGCCAAGAAGAACACCCCCGAGGAGATCGCCCTGGCCACCGTGCAGGCTC

## BLAST search

In [44]:
blast_results = aldolase.pblast(n_hits=10)

Running blast search for aldolase 1, isoform M from Drosophila melanogaster...


Fetching protein sequences: 100%|██████████| 10/10 [00:19<00:00,  1.90s/it]


In [45]:
get_nucleotide_sequences(blast_results)

Fetching nucleotide sequences: 100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


## Storing `ProteinSequence`s in a database



In [46]:
from sdrdm_database import DBConnector, create_tables

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 

>[!NOTE]
>
>If docker is not isntalled on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Connect to the database

In [47]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

### Create tables for `NucleotideSequence`

Currently, the database functionalities are under construction. Therefore, a simplified example is shown here. Instead of dumping all information of a `ProteinSequence` into the database, only the `NucleotideSequence` object is dumped into the database.

In [49]:
from pyEED.core.nucleotidesequence import NucleotideSequence

create_tables(db_connector=db, model=NucleotideSequence)

In [50]:
# Get an overview over the created tables
db.connection.tables

Tables
------
- NucleotideSequence
- regions

### Populate the database with `NucleotideSequence`s

In [51]:
# Extract only the coding sequences for all blasted protein sequences
coding_sequences = [
    sequence.coding_sequence
    for sequence in blast_results
    if sequence.coding_sequence is not None
]

In [52]:
# Insert all coding sequences into the database
db.insert(*coding_sequences, verbose=True)

Added dataset NucleotideSequence (24801bc5-62fd-432d-b5dd-c314d563be05)
Added dataset NucleotideSequence (12cee352-ee0f-468d-9666-8a34bda7d60b)
Added dataset NucleotideSequence (57c4f81d-37e4-48f7-b897-36a0b37f2a37)
Added dataset NucleotideSequence (5a8dd3e6-03ea-43b0-b61c-27dd4bcf2e49)
Added dataset NucleotideSequence (2b8a2f46-b390-4f1e-b8a7-a13ad7c0fc1a)
Added dataset NucleotideSequence (a061941e-0b82-466e-bb6c-ebb753b0b7ae)
Added dataset NucleotideSequence (13f8502f-e7f8-4893-8a22-e7cbafe9f5b5)
Added dataset NucleotideSequence (f913ab86-adda-49fb-ae8f-f0f1423757d7)
Added dataset NucleotideSequence (7c052c82-6b62-40f1-ab89-7b34f93683dd)


### Look at entries in the database

In [53]:
db.connection.table("NucleotideSequence")

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ molecule_type ┃ protein_id ┃ gene_id      ┃ sequence                                                                         ┃ id                                   ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string        │ string     │ string       │ string                                                                           │ !string                              │
├───────────────┼────────────┼──────────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────┤
│ mRNA          │ NULL       │ Ald1         │ ATGACGACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAAATCGCCCAGAAAATCGTTGCCCCCG… │ 12cee352-ee0f-468d-9666-8a34bda7d60b │
│ genomic DNA   │ NULL       │ NULL         │ ATGACCACGTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAAATTGCCCAGAAAATCGTGGCCCCCG… │ 13f8502f-e7f8-4893-8a22-e7cbafe9f5b5 │
│ mRNA          │ NULL       │ Ald-RI       │ ATGACTTTGATTGCATCAGAGGGAATTATTGCCGAGGTTTCCGTGACTTTGCTCAGTTACAAAAACATCATCGTCGAGA… │ 24801bc5-62fd-432d-b5dd-c314d563be05 │
│ mRNA          │ NULL       │ LOC6554134   │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGACATCGCCCAGCGGATCGTGGCCCCCG… │ 2b8a2f46-b390-4f1e-b8a7-a13ad7c0fc1a │
│ mRNA          │ NULL       │ LOC6617037   │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAGATCGCTCAGCGAATCGTTGCCCCCG… │ 57c4f81d-37e4-48f7-b897-36a0b37f2a37 │
│ mRNA          │ NULL       │ LOC6729730   │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGACATCGCCCAGCGAATCGTTGCCCCCG… │ 5a8dd3e6-03ea-43b0-b61c-27dd4bcf2e49 │
│ mRNA          │ NULL       │ LOC108095831 │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTCAGGGACATCGCCCAGAGGATCGTGGCCCCCG… │ 7c052c82-6b62-40f1-ab89-7b34f93683dd │
│ mRNA          │ NULL       │ LOC6538402   │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTCCGCGACATCGCCCAGCGTATCGTGGCCCCCG… │ a061941e-0b82-466e-bb6c-ebb753b0b7ae │
│ mRNA          │ NULL       │ LOC122618747 │ ATGACCACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGACATCGCCCAGCGGATCGTGGCCCCCG… │ f913ab86-adda-49fb-ae8f-f0f1423757d7 │
└───────────────┴────────────┴──────────────┴──────────────────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘

In [54]:
db.connection.table("regions")

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ start  ┃ end    ┃ note   ┃ name   ┃ cross_reference ┃ id                                   ┃ NucleotideSequence                   ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !int64 │ !int64 │ string │ string │ string          │ !string                              │ string                               │
├────────┼────────┼────────┼────────┼─────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│    320 │   1405 │ NULL   │ NULL   │ NULL            │ 40d47822-f2e1-47a1-bd5d-a2d8cdd8f5b0 │ 2b8a2f46-b390-4f1e-b8a7-a13ad7c0fc1a │
│    125 │    920 │ NULL   │ NULL   │ NULL            │ 6bcafe82-93b0-43f2-9bfa-1ecf656422a6 │ 13f8502f-e7f8-4893-8a22-e7cbafe9f5b5 │
│     22 │   1107 │ NULL   │ NULL   │ NULL            │ 97004f54-1f6a-4f44-8d57-410fb5c92b30 │ 57c4f81d-37e4-48f7-b897-36a0b37f2a37 │
│     39 │   1124 │ NULL   │ NULL   │ NULL            │ a7d466c7-d01c-4864-8101-a26df3ae55d7 │ 5a8dd3e6-03ea-43b0-b61c-27dd4bcf2e49 │
│    212 │   1396 │ NULL   │ NULL   │ NULL            │ a926b9b1-1cc4-4424-ad1a-e322ce80e7db │ 24801bc5-62fd-432d-b5dd-c314d563be05 │
│    278 │   1363 │ NULL   │ NULL   │ NULL            │ aeb7c3ba-5805-46c3-b79a-12f09e6c0772 │ 12cee352-ee0f-468d-9666-8a34bda7d60b │
│    269 │   1354 │ NULL   │ NULL   │ NULL            │ c18b917a-2aa9-4855-bd3d-6b540e3a1508 │ f913ab86-adda-49fb-ae8f-f0f1423757d7 │
│    275 │   1360 │ NULL   │ NULL   │ NULL            │ ce1984b6-a65a-42ee-abb5-b975e2f47665 │ 7c052c82-6b62-40f1-ab89-7b34f93683dd │
│   3036 │   3125 │ NULL   │ NULL   │ NULL            │ d2f01766-969f-4500-bca9-c48d92c340b1 │ 13f8502f-e7f8-4893-8a22-e7cbafe9f5b5 │
│   1345 │   1544 │ NULL   │ NULL   │ NULL            │ ea9d41ed-567f-49c5-8b2d-66b4389e320c │ 13f8502f-e7f8-4893-8a22-e7cbafe9f5b5 │
│      … │      … │ …      │ …      │ …               │ …                                    │ …                                    │
└────────┴────────┴────────┴────────┴─────────────────┴──────────────────────────────────────┴──────────────────────────────────────┘